In [66]:
import json
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import os
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from math import pi
from helpers.playlist import *
from time import sleep

In [10]:
# constants
ON_REPEAT = '37i9dQZF1Epfk0vNhDMCzw'
REPEAT_REWIND = '37i9dQZF1EpCchDHzfQcYR'
HIPHOP_NL = '0Fa3x3yr0duQJStzGpnH7w'

In [11]:
# seting up
client_id = os.environ.get('SPOT_CLIENT_ID')
secret = os.environ.get('SPOT_SECRET')

auth_manager = SpotifyClientCredentials(client_id, secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [26]:
playlist_tracks = get_playlist_tracks(sp, HIPHOP_NL)

In [63]:
ids = []

for track in playlist_tracks:
    try:
        name = track['track']['name']
        id = track['track']['id']
        ids.append(id)
    except:
        continue

In [76]:
# getting track meta and features

track_metas = []

i = 0
while True:
    try:
        tm = sp.track(ids[i])
        track_metas.append(tm)
        i += 1
        if i == len(ids):
            break
    except:
        sleep(2)

HTTP Error for GET to https://api.spotify.com/v1/tracks/6VgGIBHhiHnvhAP56rAipq with Params: {'market': None} returned 404 due to None


KeyboardInterrupt: 

In [71]:
len(track_metas)

407

In [ ]:
# data parsing
needed_metas = ['name', 'explicit', 'popularity']
needed_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
dict_data = dict((key, []) for key in (needed_metas+needed_features))

for meta, features in zip(track_metas, track_features):
    m = [meta[key] for key in needed_metas]
    f = [features[key] for key in needed_features]
    mf = m+f
    for key, value in zip(dict_data.keys(), mf):
        dict_data[key].append(value)

data = pd.DataFrame.from_dict(dict_data)
num_data = data.iloc[:,2:]

# save data
data.to_csv(path_or_buf=f'{playlist["name"]}.csv')

In [ ]:
num_data.loc[:] = MinMaxScaler().fit_transform(num_data.loc[:])

# plot size
fig=plt.figure(figsize=(12,8))

# convert column names into a list
categories=list(num_data.columns)
# number of categories
N=len(categories)

# create a list with the average of all features
value=list(num_data.mean())

# repeat first value to close the circle
# the plot is a circle, so we need to "complete the loop"
# and append the start value to the end.
value+=value[:1]
# calculate angle for each category
angles=[n/float(N)*2*pi for n in range(N)]
angles+=angles[:1]

# plot
plt.polar(angles, value)
plt.fill(angles,value,alpha=0.3)

# plt.title('Discovery Weekly Songs Audio Features', size=35)

plt.xticks(angles[:-1],categories, size=15)
plt.yticks(color='grey',size=15)
plt.show()